# L'énigme du passeur

Un brave paysan doit faire traverser la rivière sans sa petite barque à :

- sa chèvre 🐐
- son loup 🐺
- son chou 🥬

Mais... 

1. son frèle esquiffe ne peut emmener qu'un passage en plus de lui-même
2. le 🐺ne peut rester seul avec la 🐐(il la mangerait)
3. la 🐐 ne peut rester seule avec le 🥬 (elle le mangerait)


## Résolvons... à la main

1. tout le monde est sur la rive gauche : 🐺 🐐 🥬 ~~~~~~~~~
2. on fait traverser le 🥬 : 🐺 🐐 ~~~~~~~~~ 🥬 mauvaise idée puisqu'alors le 🐺 mange la 🐐
3. on fait traverser la 🐐: 🐺 🥬 ~~~~~~~~~ 🐐, c'est mieux... on peut continuer
4. on fait traverser le 🐺: 🥬 ~~~~~~~~~ 🐺 🐐, ici on est sur la rive droite, donc pas de souci
5. mais si on tente de faire traverser le 🥬, on va laisser 🐺 et 🐐 seuls...
6. on retourne donc vers le 🥬 avec la 🐐: 🥬 🐐 ~~~~~~~~~ 🐺 (on est sur la rive gauche donc pas de souci)
7. on fait traverser le 🥬 : 🐐 ~~~~~~~~~ 🐺 🥬
8. enfin on retourne chercher la 🐐 : ~~~~~~~~~ 🐺 🥬 🐐et on a terminé !


On constate une première *difficulté* : le passeur n'apparaît pas explicitement et on doit rajouter l'information de sa position pour garantir que les états présentés sont autorisés (ex. 4, le loup et la chèvre semblent seuls mais en réalité le passeur est là). Il vaudrait donc peut-être mieux modéliser aussi le passeur et le faire apparaître explicitement :

- le passeur 🙂

Et du coup les étapes présentées précédemment deviennent les suivantes, plus besoin de préciser pourquoi la configuration est interdite ou pas :
1. tout le monde est sur la rive gauche : 🐺 🐐 🥬 🙂 ~~~~~~~~~
2. on fait traverser le 🥬 : 🐺 🐐 ~~~~~~~~~ 🥬 🙂 **interdit**
3. on fait traverser la 🐐: 🐺 🥬 ~~~~~~~~~ 🐐 🙂, c'est mieux... on peut continuer
4. on fait traverser le 🐺: 🥬 ~~~~~~~~~ 🐺 🐐 🙂
5. mais si on tente de faire traverser le 🥬 : 🥬 🙂 ~~~~~~~~~ 🐺 🐐 **interdit**
6. on retourne donc vers le 🥬 avec la 🐐: 🥬 🐐 🙂 ~~~~~~~~~ 🐺
7. on fait traverser le 🥬 : 🐐 ~~~~~~~~~ 🐺 🥬 🙂
8. enfin on retourne chercher la 🐐 : ~~~~~~~~~ 🐺 🥬 🐐 🙂 et on a terminé !


## Un programme qui trouve la solution

Le but est d'écrire un programme Python qui nous donne une solution ie les différents déplacements pour atteindre l'état final : les 4 protagonistes sur la rive droite. Voyons l'algorithme réalisé pour trouver la solution :

1. On part d'un **état** initial des rives qui est `LOUP`, `CHEVRE` et `CHOU`, `PASSEUR` rive gauche, personne rive droite et que l'on peut représenter ou visualiser comme ceci :
   `🐺 🐐 🥬 🙂 ~~~~~~~~~`
2. Lorsque on a un **état**, on commence par vérifier si cet état n'est pas l'état final. Si oui, on a terminé, sinon on va tester (mentalement pour ne pas ramer pour des prunes) tous les états atteignables à partir de celui là. C'est-à-dire qu'on va essayer de faire traverser un protagoniste (de la rive gauche vers la droite ou de la droite vers la gauche) et vérifier que les règles énoncées sont respectées. De plus, il faut **mémoriser** les différents états pour ne pas rejouer toujours la même chose. 
3. Si jamais on est dans une impasse c'est-à-dire qu'on ne trouve pas d'état pour continuer à avancer et qu'on n'est pas sur un état final alors on faire marche arrière : on efface la mémoire et on choisit un autre état.

### En mode objet

Afin d'illustrer les premiers concepts de la programmation orientée objet, nous allons l'utiliser pour modéliser notre problème.

Nous allons introduire une entité (le `Passeur` ?) qui va modélisder l'`etat` des rives (qui se trouve sur la rive gauche, qui sur la rive droite)  et une `memoire` pour pouvoir changer *mentalement* d'état en faisant traverser des protagonistes. Voilà un début de code Python pour cette entité :

```python
class Passeur():
    
    def __init__(self):
        self.memoire = # une liste probablement, au début va contenir l'état initial : tout le monde à gauche
        self.etat = # pointe vers le dernier état de la mémoire
        
    def resoudre(self):
        if self.final():
            return True
        else:
            # Parcourir les états à partir de l'état courant :
                # si ce nouvel etat est viable :
                    # on essaie de continuer à résoudre :
                    # si ca se passe bien alors on retourne True
                    # sinon on revient en arrière : on oublie le nouvel état calculé
            return False # signifiant qu'on est dans une impasse
```

### Modéliser par des entiers

#### Les rives

Rive gauche - Rive droite... ouvert - fermé, ce type d'alternance se modélise bien par les entiers 0 et 1. Une des caractéristiques c'est que pour basculer d'un état à un autre il suffit de faire *1 - ...* Ainsi, si `rive` vaut 0 ou 1... pour passer à l'autre rive il suffit de faire `1 - rive`.

Nous utiliserons donc deux constantes :

In [2]:
GAUCHE = 0
DROITE = 1

#### Quant est-il des protagonistes ?

Nous pourrions manipuler des chaînes de caractères : `'loup'`, `'chèvre'` et `'chou'`... mais là encore, autant utiliser des entiers simples : 0, 1, 2, 3.

In [3]:
LOUP = 0
CHEVRE = 1
CHOU = 2
PASSEUR = 3

Cette modélisation a l'avantage de nous permettre de nous **servir du code de l'objet pour calculer sa représentation**. Ici les 4 caractères unicodes peuvent être stockés dans un tuple dans le bon ordre :

In [4]:
DESSINS = ('🐺', '🐐', '🥬', '🙂')

Ainsi `DESSINS[LOUP]` me donne le caractère qui représente le loup. 

#### Règle importante

Séparer la modélisation de la représentation ou visualisation est une chose très importante. 

D'autre part, nous pouvons aussi nous servir de cet ordre pour modéliser l'état des rives la liste : `[GAUCHE, GAUCHE, DROITE, DROITE]` signifie que le personnage ayant l'identifiant 0 (ie le loup) se trouve sur la rive gauche, ainsi que la chèvre, le chou lui est sur la rive droite avec le passeur. Nous pouvons affiner notre classe `Passeur` et nous profitons pour ajouter la méthode qui détermine qu'on a atteint l'état final :

```python

PROTAGONISTES = (LOUP, CHEVRE, CHOU, PASSEUR)

class Passeur():
    
    def __init__(self):
        self.memoire = [[GAUCHE] * len(PROTAGONISTES)]
        self.etat = self.memoire[-1]
        
    def final(self):
        return self.etat == [DROITE] * len(PROTAGONISTES)
```

### Passer à l'état suivant

Un état est donc une simple liste de 4 éléments de la forme `[x, y, z, t]` où chacune des variables vaut 0 ou 1. 0 signifiant que le personnage concerné est à GAUCHE, et 1 qu'il est à DROITE. Plus précisemment `x` représente la rive du loup, `y` celle de la chèvre, `z` celle du chou et `t` celle du passeur. Pour passer à un autre état, il suffit de sélectionner un personnage à faire changer de rive. Par exemple, on passe de `[0, 0, 0, 0]` à `[1, 0, 0, 0]` en changeant le loup de rive.

Ainsi le `# Parcourir les états à partir de l'état courant :` de notre algorithme se traduit bien par une simple boucle `for` :

```python
for protagoniste in PROTAGONISTES:
    # calculer nouvel état 
```

Et *passer à l'état suivant* consiste à récupérer une copie de l'état courant et d'y faire traverser un protagoniste, c'est-à-dire faire une copie de l'état courant (le dernier mémorisé) dans `self.suivant` puis changer la valeur de la rive pour l'identifiant concerné. De plus, il faut aussi faire traverser le passeur si jamais ce-dernier n'est pas le protagoniste.

```python 
def traverse(self, protagonoiste):
    self.suivant = self.etat.copy()
    self.suivant[protagoniste] = 1 - self.suivant[protagoniste]
    if protagoniste != PASSEUR:
        self.suivant[PASSEUR] = 1 - self.suivant[PASSEUR]
```

Et dans le même esprit, chaque action élémentaire de notre problème doit se modéliser **simplement** avec notre modèle :

### Détecter le danger

Une fois l'état suivant calculé, il faut que le `Passeur` puisse identifer une situation de danger. Il y a danger si la chèvre et le loup sont du même côté, sans le passeur ou si la chèvre et le chou sont du même côté toujours sans le passeur. Ce qui nous donne avec notre modèle :

```python
def danger(self):
    loup_mange_chevre = self.suivant[CHEVRE] == self.suivant[LOUP] and self.suivant[LOUP] != self.suivant[PASSEUR]
    chevre_mange_chou = self.suivant[CHEVRE] == self.suivant[CHOU] and self.suivant[CHEVRE] != self.suivant[PASSEUR]
    return loup_mange_chevre or chevre_mange_chou
```

### Mémoriser

Cela consiste à ajouter une copie de `suivant` dans la `memoire` :

```python
def memoire(self):
    self.memoire.append(self.suivant.copy())
```

### Déjà vu ?

Un état est déjà vu s'il fait partie de notre mémoire :

```python
def deja_vu(self):
    return self.suivant in self.memoire
```

### Avancer ou continuer...

Il s'agit de continuer à résoudre ie faire l'appel récursif. Si cet appel retourne `True` alors on retourne `True` sinon on doit oublier notre dernier état pour que la boucle continue avec un nouvel. Si on épuise toute la boucle sans renvoyer `True` c'est que c'est une impasse : on retourne `False`.

```python
for protagoniste in PROTAGONISTES:
    self.traverse(protagoniste)
    if not self.danger() and not self.deja_vu():
        self.memorise()
        if self.resoudre():
            return True
        else:
            self.oublie()
return False
```

### Oublier

Il s'agit de repositionner l'`etat` sur l'avant-dernier de la mémoire et de retirer le dernier élément.

```python
def oublie(self):
    self.memoire.pop()
    self.etat = self.memoire[-1]
```

### Afficher la solution

Une fois la résolution faite, afficher la solution va consister à parcourir la mémoire et à construire pour chaque configuration rencontrée deux chaines de caractères : la première avec les `DESSINS` des protagonistes qui se trouvent sur la rive `GAUCHE` et une 2e avec ceux de la rive droite.

```python
def afficher_solution(self):
    for config in self.memoire:
        rives = ['', '']
        for protagoniste in PROTAGONISTES:
            rives[config[protagoniste]] += DESSINS[protagoniste]
        print(rives[GAUCHE] + ' ~~~~~~~~~ ' + rives[DROITE])
```

## Le code complet

In [9]:
GAUCHE = 0
DROITE = 1

LOUP = 0
CHEVRE = 1
POMME = 2
PASSEUR = 3
PROTAGONISTES = (LOUP, CHEVRE, POMME, PASSEUR)

DESSINS = ('🐺', '🐐', '🍏', '🙂')

class Passeur():
    def __init__(self):
        self.suivant = None
        self.memoire = [[GAUCHE] * len(PROTAGONISTES)]
        self.etat = self.memoire[-1]


    def affiche_solution(self):
        for config in self.memoire:
            rives = ['', '']
            for protagoniste in PROTAGONISTES:
                rives[config[protagoniste]] += DESSINS[protagoniste]
            print(rives[GAUCHE] + ' ~~~~~~~~~ ' + rives[DROITE])
        
    def fini(self):
        return self.etat == [DROITE] * len(PROTAGONISTES)
    
    def memorise(self):
        self.memoire.append(self.suivant.copy())
        self.etat = self.memoire[-1]

    def oublie(self):
        self.memoire.pop()
        self.etat = self.memoire[-1]
        
    def deja_vu(self):
        return self.suivant in self.memoire
    
    def danger(self):
        loup_mange_chevre = self.suivant[CHEVRE] == self.suivant[LOUP] and self.suivant[LOUP] != self.suivant[PASSEUR]
        chevre_mange_chou = self.suivant[CHEVRE] == self.suivant[CHOU] and self.suivant[CHEVRE] != self.suivant[PASSEUR]
        return loup_mange_chevre or chevre_mange_chou
    
    def traverse(self, protagoniste):
        self.suivant = self.etat.copy()
        self.suivant[protagoniste] = 1 - self.suivant[protagoniste]
        if protagoniste != PASSEUR:
            self.suivant[PASSEUR] = 1 - self.suivant[PASSEUR]
        
    def resoud(self):
        if self.fini():
            return True
        else:
            for protagoniste in PROTAGONISTES:
                self.traverse(protagoniste)
                if not self.danger() and not self.deja_vu():
                    self.memorise()
                    if self.resoud():
                        return True
                    else:
                        self.oublie()
            return False
                
seb = Passeur()
if seb.resoud():
    seb.affiche_solution()
else:
    print('Pas de solution')

🐺🐐🍏🙂 ~~~~~~~~~ 
🐺🍏 ~~~~~~~~~ 🐐🙂
🐺🍏🙂 ~~~~~~~~~ 🐐
🍏 ~~~~~~~~~ 🐺🐐🙂
🐐🍏🙂 ~~~~~~~~~ 🐺
🐐 ~~~~~~~~~ 🐺🍏🙂
🐐🙂 ~~~~~~~~~ 🐺🍏
 ~~~~~~~~~ 🐺🐐🍏🙂


## Vos commentaires